In [20]:
import os
import keras
import numpy as np 
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import plotly.express as px
from keras import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import resnet

In [21]:
import zipfile

zip_file_path = r'monkeypox_data.zip'
dest_path = r'data'

with zipfile.ZipFile(zip_file_path, 'r') as z:
    z.extractall(dest_path)


In [22]:
info_file = 'data/Monkeypox_Dataset_metadata.csv'
info = pd.read_csv(info_file)
info.head()

,ImageID,Label
0,M01_01,Monkeypox
1,M01_02,Monkeypox
2,M01_03,Monkeypox
3,M01_04,Monkeypox
4,M02_01,Monkeypox


In [23]:
fig = px.pie(
    names=info.Label.unique(),
    values=info.Label.value_counts(),
    hole=0.2,
    width=500,
    height=500
)
fig.update_layout({'title':{'text':'Class Distribution','x':0.45}})
fig.show()

In [14]:
# Data Generators
train_gen = ImageDataGenerator(rescale=1./255)
valid_gen = ImageDataGenerator(rescale=1./255,validation_split=0.4)

# Loading Data
train_data = train_gen.flow_from_directory("Augmented Images\Augmented Images",target_size=(256,256),shuffle=True,class_mode='binary')
valid_data = valid_gen.flow_from_directory('Original Images\Original Images',target_size=(256,256),shuffle=True,subset='training',class_mode='binary')
test_data = valid_gen.flow_from_directory('Original Images\Original Images',target_size=(256,256),shuffle=True,subset='validation',class_mode='binary')

Found 3192 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [15]:
# Model Structure : 
with tf.device('/GPU:0'):
  base_model = resnet.ResNet50(
      include_top=False,
      input_shape=(256,256,3),
      weights='imagenet'
  )
  base_model.trainable = False

  model = Sequential([
      base_model,
      GlobalAveragePooling2D(),
      Dense(256,activation='relu'),
      BatchNormalization(),
      Dense(164,activation='relu'),
      BatchNormalization(),
      Dense(1,activation='sigmoid')
  ])

  model.compile(
      loss='binary_crossentropy',
      optimizer='Adam',
      metrics=['accuracy']
  )

  cb = [EarlyStopping(patience=5,monitor='val_accuracy',mode='max',restore_best_weights=True),ModelCheckpoint("ResNet50V2-01.h5",save_best_only=True)]

  model.fit(
      train_data,
      epochs=50,
      validation_data=valid_data,
      callbacks=cb
  )

Epoch 1/50
100/100 [==============================] - 206s 2s/step - loss: 0.6093 - accuracy: 0.6924 - val_loss: 0.8235 - val_accuracy: 0.4493
Epoch 2/50
100/100 [==============================] - 193s 2s/step - loss: 0.5398 - accuracy: 0.7253 - val_loss: 0.5564 - val_accuracy: 0.7246
Epoch 3/50
100/100 [==============================] - 192s 2s/step - loss: 0.5321 - accuracy: 0.7372 - val_loss: 1.4747 - val_accuracy: 0.4493
Epoch 4/50
100/100 [==============================] - 187s 2s/step - loss: 0.5177 - accuracy: 0.7500 - val_loss: 3.3649 - val_accuracy: 0.4493
Epoch 5/50
100/100 [==============================] - 190s 2s/step - loss: 0.4739 - accuracy: 0.7766 - val_loss: 0.9853 - val_accuracy: 0.7319
Epoch 6/50
100/100 [==============================] - 186s 2s/step - loss: 0.4672 - accuracy: 0.7807 - val_loss: 2.7715 - val_accuracy: 0.4493
Epoch 7/50
100/100 [==============================] - 181s 2s/step - loss: 0.4389 - accuracy: 0.7932 - val_loss: 1.6546 - val_accuracy: 0.4638

In [18]:
# At Last
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 256)               524544    
                                                                 
 batch_normalization_4 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 164)               42148     
                                                                 
 batch_normalization_5 (Batc  (None, 164)             

In [25]:
model = keras.models.load_model('Monkepox_ResNet50_acc-100.h5') # Training Accuracy : 99.0% Validation Accuracy : 100.0%
model.evaluate(test_data)
print("Test Accuracy : ",model.evaluate(test_data)[1]*100,"%")
print("Test Loss : ",model.evaluate(test_data)[0]*100,"%")

3/3 [==============================] - 4s 1s/step - loss: 0.0056 - accuracy: 1.0000
Test Accuracy :  100.0 %
3/3 [==============================] - 4s 1s/step - loss: 0.0056 - accuracy: 1.0000
Test Loss :  0.5589955486357212 %
